<a href="https://colab.research.google.com/github/hernansalinas/autogrades/blob/main/Actividades_clase/Actividad_09_MatricesAutovaloresAutovectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
#import torch as pt
import sympy as sp

# Autovalores, autovectores, diagonalización

Una matriz es diagonalizable cuando es semejante a una matriz diagonal. Esto significa que mediante un cambio de base, podemos reducir la matriz original a una forma diagonal. Para que una matriz sea diagonalizable, debe cumplir las siguientes condiciones:

**Autovectores linealmente independientes**: Una matriz $(A \in \mathbb{R}^{n \times n})$ es diagonalizable si y solo si tiene (n) autovectores linealmente independientes. Esto significa que los autovectores asociados a los autovalores deben ser linealmente independientes.

**Semejanza**: Existe una matriz invertible (P) tal que ($P^{-1}AP = D$), donde (D) es una matriz diagonal formada por los autovalores de (A).

La diagonalización es un proceso que transforma una matriz cuadrada ( A ) en una matriz diagonal ( D ) mediante una matriz invertible ( P ), tal que:

$P^{-1}AP = D$

La matriz diagonal ( D ) contiene los autovalores de ( A ) en la diagonal principal, y las columnas de ( P ) son los autovectores correspondientes a esos autovalores.

## Numpy

In [2]:
# Definir una matriz 3x3
A=np.array([[1,3,3],
              [-3,-5,-3],
              [3,3,1]])

# Calcular autovalores y autovectores
autovalores, autovectores = np.linalg.eig(A)
U = autovectores.copy()
print("Autovalores:", autovalores)
print("Autovectores:\n", autovectores)


Autovalores: [ 1. -2. -2.]
Autovectores:
 [[ 5.77350269e-01 -1.28197512e-16 -6.72654054e-01]
 [-5.77350269e-01 -7.07106781e-01  7.37141689e-01]
 [ 5.77350269e-01  7.07106781e-01 -6.44876349e-02]]


### Matriz Diagonal

In [3]:
D = np.linalg.inv(U)@A@U
np.set_printoptions(suppress=True, precision=6)
D

array([[ 1., -0., -0.],
       [ 0., -2.,  0.],
       [ 0.,  0., -2.]])

In [4]:
np.set_printoptions(suppress=True, precision=6)

## Actividad 1

Construir un programa que permita elevar una matriz a una potencia.


La exponencial de una matriz ( $e^A$) se define como la serie de Taylor de la función exponencial evaluada en la matriz ( A ):


\begin{equation}
e^A = I + A + \frac{1}{2!}A^2 + \frac{1}{3!}A^3 + \cdots = \sum_{n=0}^{\infty} \frac{1}{n!}A^n
\end{equation}


Donde ( $e^D$ ) es simplemente la matriz exponencial de la matriz diagonal ( D ), que es fácil de calcular porque solo implica elevar los autovalores de (A) a la potencia de (e):



\begin{equation}
e^A = Pe^DP^{-1}
\end{equation}


Exponencial de una matriz diagonal

\begin{equation}
e^D = \begin{bmatrix}
e^{\lambda_1} & 0 & \cdots & 0 \\
0 & e^{\lambda_2} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & e^{\lambda_n}
\end{bmatrix}
\end{equation}

In [30]:
A=sp.Matrix([[5,-8,1],[0,0,7],[0,0,-2]])
A

Matrix([
[5, -8,  1],
[0,  0,  7],
[0,  0, -2]])

In [31]:
λ,x,y,z=sp.symbols('λ,x,y,z')

Aλ_I=A-λ*sp.eye(len(A[0,:]))                    #A-λI
Apol=sp.det(Aλ_I)                               #polinomio característico

root=sp.roots(Apol)                             #autovalores
roots=np.zeros(len(A[0,:]))
for i,j in enumerate(root):                     #este ciclo es para crear la matriz D, cuando A tiene
    roots[i]=j                                  #valores propios con multiplicidad mayor que 1
    if root[j]>1:
        for _ in range(root[j]):
            roots[i+_]=j

D=np.diag(roots)                                 #matriz diagonal

A_=sp.lambdify(λ,Aλ_I,'sympy')                  #A-λI para evaluar
v=sp.Matrix([[x],[y],[z]])

for i in range(len(A[0,:])):
    sis=A_(roots[i])@v
    print(f'Autovector para λ={roots[i]}: {sp.solve(sis)}')
D

Autovector para λ=5.0: {y: 0.0, z: 0.0}
Autovector para λ=-2.0: {x: -4.14285714285714*z, y: -3.5*z}
Autovector para λ=0.0: {x: 1.6*y, z: 0.0}


array([[ 5.,  0.,  0.],
       [ 0., -2.,  0.],
       [ 0.,  0.,  0.]])

Hasta acá llegó mi intento de hacerlo todo desde cero, esos vectores propios que me dan a mí tienen como una variable libre o algo así, porque habría que devolverse hacia atrás sustituyendo. Además no supe como sacar los vectores para seguir usándolos de manera paramétrica, es decir, cuando lo meta cualquier otra matriz y no tenga que meter los autovectores manualmente. Entonces voy a hacerlo con librerías. Cabe resaltar que el procedimiento sí está bueno, es sólo por cuestiones de generalizar el algoritmo.

In [33]:
vals=A.eigenvals()
vects=A.eigenvects()

roots=np.zeros(len(A[0,:]))
for i,j in enumerate(vals):
    roots[i]=j
    if root[j]>1:
        for _ in range(root[j]):
            roots[i+_]=j

D=np.diag(roots)

#vects[_][0]       valor
#vects[_][1]       multiplicidad
#vects[_][2][0]    vector

v1=vects[0][2][0]
v2=vects[1][2][0]
v3=vects[2][2][0]

P=v1.row_join(v2).row_join(v3)
Pinv=P.inv()

Diagonalización de A,

In [34]:
P@D@Pinv

Matrix([
[5.0, -8.0,  1.0],
[  0,    0,  7.0],
[  0,    0, -2.0]])

Exponencial de A,

In [37]:
expD=np.exp(D)
P@expD@Pinv

Matrix([
[145.870301959719, -234.935340278408, -215.914543314714],
[            -2.5,               1.5,   -4.580816348471],
[             1.0,              -0.6,  2.17819242609376]])

## Autovalores y autovectores complejos

In [8]:
theta=-np.pi/4
A = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta), np.cos(theta)]])

# Calcular autovalores y autovectores
autovalores, autovectores = np.linalg.eig(A)
U = autovectores.copy()
print("Autovalores:", autovalores)
print("Autovectores:\n", autovectores)

D = np.linalg.inv(U)@A@U
np.set_printoptions(suppress=True, precision=6)
D

Autovalores: [0.707107+0.707107j 0.707107-0.707107j]
Autovectores:
 [[0.      -0.707107j 0.      +0.707107j]
 [0.707107+0.j       0.707107-0.j      ]]


array([[0.707107+0.707107j, 0.      +0.j      ],
       [0.      -0.j      , 0.707107-0.707107j]])

In [9]:
np.exp(D)

array([[1.541863+1.317538j, 1.      +0.j      ],
       [1.      -0.j      , 1.541863-1.317538j]])

## Actividad 2

Considere la ecuación cuadrática $x^2-4xy+3y^2=6$, esta ecuación cuadrática se corresponde a una hiperbola, la cual puede ser escrita en forma matricial como sigue:

$A\vec{v} \cdot \vec{v}=6$, donde $\vec{v}=(x, y)$ y A es una matrix simetrica,

\begin{equation}
A=\begin{bmatrix}
1 & -2 \\
-2 & 3 \\
\end{bmatrix}
\end{equation}

1. Realice un grafico que represente la hyperbola







- Diagonalice A, para ello construya la matrix Q de autovectores ortoganales
tal que $D=Q^{T}AQ$, en este caso se dice que A es diagonalizable ortogonalmente, donde $A=QDQ^T$.

- Aplique el resultado de la siguiente demostración para encontrar una nueva hyperbola en una  nueva base

\begin{equation}
QDQ^T \vec{v} \cdot \vec{v} = d
\end{equation}


\begin{equation}
Q(DQ^T\vec{v}) \cdot \vec{v} = (DQ^T\vec{v}) \cdot (Q^T\vec{v})=d
\end{equation}


\begin{equation}
(DQ^T\vec{v}) \cdot (Q^T\vec{v})=d
\end{equation}


Sea $u=Q^T\vec{v}$

Luego :


\begin{equation}
(D\vec{u}) \cdot \vec{u}=d
\end{equation}

donde $\vec{u}=(x', y')$, tal que $D\vec{u} \cdot \vec{u}=d$ es una nueva hyperbola









In [43]:
A=sp.Matrix([[1,-2],[-2,3]])
A

Matrix([
[ 1, -2],
[-2,  3]])

In [54]:
vals=A.eigenvals()
vects=A.eigenvects()

roots=np.zeros(len(A[0,:]))
for i,j in enumerate(vals):
    roots[i]=j

D=np.diag(roots)

v1=vects[0][2][0]
v2=vects[1][2][0]

Q=v1.row_join(v2)
Q[:,0].dot(Q[:,1])

(1/2 - sqrt(5)/2)*(1/2 + sqrt(5)/2) + 1

### Sympy

In [10]:
from sympy import Matrix, simplify

# Definir una matriz 3x3
A = Matrix([[4, 2, 0],
            [2, 4, 1],
            [0, 1, 3]])

# Calcular autovalores y autovectores
autovalores = A.eigenvals()
autovectores = A.eigenvects()

# Simplificar autovalores y autovectores
autovalores_simplificados = {simplify(l): m for l, m in autovalores.items()}
autovectores_simplificados = [(simplify(l), m, [simplify(v) for v in vs]) for l, m, vs in autovectores]

print("Autovalores simplificados:", autovalores_simplificados)
print("Autovectores simplificados:\n", autovectores_simplificados)


Autovalores simplificados: {11/3 + 16*2**(1/3)/(3*(61 + 3*sqrt(1407)*I)**(1/3)) + 2**(2/3)*(61 + 3*sqrt(1407)*I)**(1/3)/6: 1, (-128*2**(1/3) + (1 - sqrt(3)*I)*(44 + 2**(2/3)*(-1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(61 + 3*sqrt(1407)*I)**(1/3))/(12*(1 - sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3)): 1, (-128*2**(1/3) + (1 + sqrt(3)*I)*(44 - 2**(2/3)*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(61 + 3*sqrt(1407)*I)**(1/3))/(12*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3)): 1}
Autovectores simplificados:
 [((-128*2**(1/3) + (1 + sqrt(3)*I)*(44 - 2**(2/3)*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(61 + 3*sqrt(1407)*I)**(1/3))/(12*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3)), 1, [Matrix([
[(12*(-176 + 7*2**(2/3)*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(1 + sqrt(3)*I)**2*(61 + 3*sqrt(1407)*I) + (61 + 3*sqrt(1407)*I)**(1/3)*(128*2**(1/3) + (-44 + 2**(2/3)*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))**2 + 10752*2**(1/3)*(1 + sqrt(3)

In [11]:
autovalores_simplificados

{11/3 + 16*2**(1/3)/(3*(61 + 3*sqrt(1407)*I)**(1/3)) + 2**(2/3)*(61 + 3*sqrt(1407)*I)**(1/3)/6: 1,
 (-128*2**(1/3) + (1 - sqrt(3)*I)*(44 + 2**(2/3)*(-1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(61 + 3*sqrt(1407)*I)**(1/3))/(12*(1 - sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3)): 1,
 (-128*2**(1/3) + (1 + sqrt(3)*I)*(44 - 2**(2/3)*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3))*(61 + 3*sqrt(1407)*I)**(1/3))/(12*(1 + sqrt(3)*I)*(61 + 3*sqrt(1407)*I)**(1/3)): 1}

# Pytorch

Definción de una matrix de tres formas:
Hasta ahora sabemos definir matrices como objetos de numpy y objetos de sympy, en la actualidad, el rápido crecimiento de las unidades de procesamiento gráfico GPU,  permite definir otro tipo de objetos que pueden acelerar el computo computacional.


- Como objeto de numpy
- Como objeto de sympy
- Como objeto de pytorch


La velocidad de ejecución entre PyTorch y Numpy puede variar dependiendo del tamaño de los datos y si se está utilizando o no la aceleración por GPU. Para operaciones con matrices pequeñas y cálculos que no requieren de la paralelización de una GPU, Numpy puede ser más rápido porque opera directamente en la CPU y la memoria principal. Sin embargo, para matrices grandes y operaciones que pueden ser paralelizadas, PyTorch con soporte de GPU puede ser significativamente más rápido.

Comparativa de comandos de ejecución entre pytorch, numpy  y sympy


| Operación | PyTorch | Numpy | SymPy |
|-----------|---------|-------|-------|
| Creación de una matriz | `torch.tensor([[1, 2], [3, 4]])` | `np.array([[1, 2], [3, 4]])` | `Matrix([[1, 2], [3, 4]])` |
| Transposición de una matriz | `torch.transpose(A, 0, 1)` | `A.T` o `np.transpose(A)` | `A.T` |
| Multiplicación de matrices | `torch.matmul(A, B)` | `np.dot(A, B)` o `A @ B` | `A * B` |
| Inversa de una matriz | `torch.inverse(A)` | `np.linalg.inv(A)` | `A**-1` |
| Determinante de una matriz | `torch.det(A)` | `np.linalg.det(A)` | `A.det()` |
| Autovalores y autovectores | `torch.eig(A, eigenvectors=True)` | `np.linalg.eig(A)` | `A.eigenvals()`, `A.eigenvects()` |



Ejemplo básico del comparativo de un problema de algebra lineal, rotación de un vector:

In [12]:

def matrixA(theta, tipo="numpy"):
  if(tipo=="np"):
    return np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta), np.cos(theta)]])
  elif(tipo=="sp"):
    return np.array([[sp.cos(theta),-sp.sin(theta)],[sp.sin(theta), sp.cos(theta)]])

  elif(tipo=="pt"):
    return pt.tensor([[np.cos(theta),-np.sin(theta)],[np.sin(theta), np.cos(theta)]])

  else:

    print("Define el tipo de variable que desea")


In [13]:
x = sp.Symbol("x")
M = sp.Matrix(matrixA(np.pi/4, tipo="sp"))
M

Matrix([
[0.707106781186548, -0.707106781186547],
[0.707106781186547,  0.707106781186548]])

In [14]:
x=np.pi/4
M = matrixA(x, tipo="np")
M

array([[ 0.707107, -0.707107],
       [ 0.707107,  0.707107]])

In [ ]:
matrixA(x, tipo="pt")
M

## Actividad 3
Resolver el problema de la potencia de una matriz empleando pytorch, construir.